<a href="https://colab.research.google.com/github/acdc2019/algo-trading/blob/main/python/notebooks/strategies/rsi_strategy/RSIStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RSI Strategy**
## **15 mins strategy**
* Step 1: A 15 min candle has to close above 70 RSI value.
* Step 2: Above High of this candle Buy signal is generated.
* Step 3: Stop Loss of the trade has to be below the earlier 15 min candles low.
* Step 4: Volume of the candle in which buy signal is generated has to be atleast 2 to 3 times of earlier 5 candles (***Q: average of earlier 5 candle volume?***)
* Step 5: Hourly candles RSI should be above 50 for confirmation.
* Step 6: In the last 5 candles the stock movement should not be more than 4% of the day movement
* Step 7: If Stop Loss of the candle is less than 6000 INR then the trade should be executed

#### **Strategy Parameters**
* window_start, window_end: Dates between which to look for signal
* rsi_15min = 70
* rsi_60min = 50
* stop_loss = 6000
* back_candles = 5
* volume_multiple = 2 or 3 times
* daily_movement_pct = 4

Install libs

In [1]:
!pip install ta
!pip install pandas==1.3.5
!pip install plotly
!pip install numpy

  Created wheel for ta: filename=ta-0.9.0-py3-none-any.whl size=28908 sha256=424113dea4f0550e019409d94ec11e8e1877a0c9371a70ae3ecd7a44d9fe80f5
  Stored in directory: /root/.cache/pip/wheels/72/78/64/cc1c01506a1010a9845e9bd7c69333730f7174661228ea4f98
Successfully built ta
     |████████████████████████████████| 11.3 MB 4.3 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.


In [ ]:
!wget https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=GHSAT0AAAAAABQZJYETEQYUGLNGHAJZQ7CAYPP566Q -O helpers.py

--2022-01-25 12:39:39--  https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=GHSAT0AAAAAABQZJYETEQYUGLNGHAJZQ7CAYPP566Q
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... ^C


Import Libs

In [1]:
import pandas as pd
from ta.momentum import RSIIndicator
from ta.trend import ADXIndicator
from plotly.subplots import make_subplots
from datetime import date
import numpy as np

In [2]:
class Signal():
    def __init__(self, strategy: str, sym: str, lot_size: int, ts: date, entry_price: float, stop_loss: float,
                 rsi: float, hourly_rsi: float, vol: int, mean_vol: int, daily_mov_pct: float) -> None:
        self.strategy = strategy
        self.sym = sym
        self.lot_size = lot_size
        self.ts = ts
        self.entry_price = entry_price
        self.stop_loss = stop_loss
        self.rsi = rsi
        self.hourly_rsi = hourly_rsi
        self.vol = vol
        self.mean_vol = mean_vol
        self.daily_mov_pct = daily_mov_pct
        self.pnl = 0
        self.comment = ""

    def __str__(self) -> str:
        return "Strategy: {}, Sym: {}, TS: {}, Entry: {}, StopLoss: {}, PnL: {}, Comment: {}".format(
            self.strategy, self.sym, self.ts, self.entry_price, self.stop_loss, self.pnl, self.comment)

class BackTestResult():
    def __init__(self, sym: str, signals: list) -> None:
        self.signals = signals
        self.sym = sym
        self.total_pnl = self._calc_total_pnl()
        self.gross_profit = self._calc_gross_profit()
        self.gross_loss = self._calc_gross_loss()
        self.profit_factor = self._calc_profit_factor()

    def _calc_profit_factor(self) -> float:
        gross_profit = self._calc_gross_profit()
        gross_loss = abs(self._calc_gross_loss())
        if (gross_loss == 0):
            gross_loss = 0.1

        profit_factor = gross_profit/gross_loss
        return round(profit_factor, 2)

    def _calc_gross_profit(self) -> float:
        gross_profit = 0
        for signal in self.signals:
            if (signal.pnl > 0):
                gross_profit = gross_profit + signal.pnl

        return round(gross_profit, 2)

    def _calc_gross_loss(self) -> float:
        gross_loss = 0
        for signal in self.signals:
            if (signal.pnl < 0):
                gross_loss = gross_loss - signal.pnl

        return round(gross_loss, 2)

    def _calc_strike_rate(self) -> float:
        if (len(self.signals) == 0):
            return 0

        profit = 0
        notional = 0
        for signal in self.signals:
            if (signal.pnl > 0):
                profit = profit+signal.pnl

            notional = notional + abs(signal.pnl)

        return round(profit/notional, 2)

    def _calc_total_pnl(self) -> float:
        total_pnl = 0
        for signal in self.signals:
            total_pnl = total_pnl + signal.pnl

        return round(total_pnl, 2)

    def __str__(self) -> str:
        return 'Sym,{},Total PnL,{},ProfitFactor,{},GrossProfit,{},GrossLoss,{},Total Signals,{}'.format(
            self.sym, self.total_pnl, self.profit_factor, self.gross_profit, self.gross_loss, len(self.signals))


In [3]:
def get_previous_candles(df: pd.DataFrame, index, n: int, include_index=False):
    '''
    Returns previous n candles from the given index in the DataFrame
    Parameters:
    df (DataFrame): DataFrame from which to return the previous candles
    index (DataFrame Index): DataFrame Index from which to return the previous candles
    n (int): Number of previous candles to return from index
    include_index (bool): If current index should be included in returned DataFrame
    Returns:
    DataFrame: Pandas dataframe with the previous n candles
    '''
    loc = df.index.get_loc(index)
    fromIdx = loc-n
    toIdx = loc+1 if include_index else loc
    return df.iloc[fromIdx:toIdx]


def get_next_candles(df: pd.DataFrame, index, n: int):
    '''
    Returns next n candles from the given index in the DataFrame
    Parameters:
    df (DataFrame): DataFrame from which to return the next candles
    index (DataFrame Index): DataFrame Index from which to return the next candles
    n (int): Number of next candles to return from index
    Returns:
    DataFrame: Pandas dataframe with the next n candles
    '''
    loc = df.index.get_loc(index)
    return df.iloc[loc+1:loc+1+n]


def get_hourly_df(df: pd.DataFrame, index) -> pd.DataFrame:
    df_temp = df[:index]
    # print(index)
    # print(df_15min_temp.tail(10))
    df_60min_o = df_temp['Open'].resample(
        '60Min', offset='30Min').apply({'Open': 'first'})
    df_60min_h = df_temp['High'].resample(
        '60Min', offset='30Min').apply({'High': 'max'})
    df_60min_l = df_temp['Low'].resample(
        '60Min', offset='30Min').apply({'Low': 'min'})
    df_60min_c = df_temp['Close'].resample(
        '60Min', offset='30Min').apply({'Close': 'last'})
    df_60min_vol = df_temp['Volume'].resample(
        '60Min', offset='30Min').apply({'Volume': 'sum'})

    df_60min = pd.concat([df_60min_o, df_60min_h, df_60min_l,
                         df_60min_c, df_60min_vol], axis=1)
    df_60min.dropna(subset=['Open'], inplace=True)

    rsi = RSIIndicator(df_60min['Close']).rsi()
    df_60min = df_60min.assign(rsi=rsi.values)
    return df_60min


def get_tanaji_pct(df: pd.DataFrame, index, n: int, high: float) -> float:
    prev_candles = get_previous_candles(df, index, n)
    min_low = prev_candles['Low'].min()
    daily_movement = high - min_low
    tanaji_pct = (daily_movement/min_low)*100
    return tanaji_pct


def get_sell_tanaji_pct(df: pd.DataFrame, index, n: int, low: float) -> float:
    prev_candles = get_previous_candles(df, index, n)
    max_high = prev_candles['High'].max()
    daily_movement = max_high - low
    tanaji_pct = (daily_movement/max_high)*100
    return tanaji_pct


def generate_buy_signal_pnl(signal: Signal, df: pd.DataFrame, stop_gain: int) -> None:
    df = df[signal.ts:].copy()
    df.drop(index=df.index[0], axis=0, inplace=True)
    rnum = 0
    stop_loss_gap = signal.entry_price - signal.stop_loss
    rsi_breached = False
    adx = 0
    prev_adx = 0
    adx_breached = False

    for index, row in df.iterrows():
        adx = row['adx']
        if (row['Low'] < signal.stop_loss):
            #print('rnum: {}, index: {}, low: {}, sl: {}'.format(rnum, index, row['Low'], signal.stop_loss))
            signal.pnl = -1 * (signal.entry_price -
                               signal.stop_loss) * signal.lot_size
            signal.comment = "Stop Loss Breached at {}".format(index)
            break

        if ((row['High'] - signal.entry_price) * signal.lot_size >= stop_gain):
            signal.pnl = stop_gain
            signal.comment = "Stop Gain reached at {}".format(index)
            break

        if (rsi_breached == True):
            signal.pnl = (row['Open'] - signal.entry_price) * signal.lot_size
            signal.comment = "RSI Breached at {}".format(index)
            break
        
        if (adx_breached == True and prev_adx - adx >= 0.25):
            adx_breached = False
            signal.pnl = (row['Open'] - signal.entry_price) * signal.lot_size
            signal.comment = "ADX Breached at {}".format(index)
            break

        if (prev_adx != 0 and prev_adx - adx >= 0.25 ):
            adx_breached = False

        if (row['High'] - signal.stop_loss > 2*stop_loss_gap):
            # print('New Stoploss: {}, Old Stoploss: {}, Stoploss gap: {}'.format(                row['High'] - stop_loss_gap, signal.stop_loss, stop_loss_gap))
            # signal.stop_loss = row['High'] - stop_loss_gap
            signal.stop_loss = signal.stop_loss

        '''if (row['rsi'] < 70 and row['Volume'] >= 2*row['vol_sma5']):
            rsi_breached = True'''

        if(rnum > 50):
            signal.pnl = (row['Close'] -
                          signal.entry_price) * signal.lot_size
            # print('Signal failed to generate PnL: {}'.format(signal.pnl))
            signal.comment = "Signal failed to generate {}".format(index)
            break

        rnum = rnum+1
        prev_adx = adx


def generate_sell_signal_pnl(signal: Signal, df: pd.DataFrame, stop_gain: float) -> None:
    df = df[signal.ts:].copy()
    df.drop(index=df.index[0], axis=0, inplace=True)
    rnum = 0
    for index, row in df.iterrows():
        if (row['High'] > signal.stop_loss):
            #print('rnum: {}, index: {}, low: {}, sl: {}'.format(rnum, index, row['Low'], signal.stop_loss))
            signal.pnl = (signal.entry_price -
                          signal.stop_loss) * signal.lot_size
            break

        if ((signal.entry_price - row['Low']) * signal.lot_size >= stop_gain):
            signal.pnl = stop_gain
            break

        if(rnum > 50):
            signal.pnl = (signal.entry_price - row['Close']) * signal.lot_size
            # print('Signal failed to generate PnL: {}'.format(signal.pnl))
            break
        rnum = rnum+1

## Load 15 min and 60 min stock data and calculate RSI

In [45]:
file_15min = '/content/CADILAHC-HIST.csv'

df_15min = pd.read_csv(file_15min, parse_dates=['Date'], index_col=['Date'])

# Add RSI
rsi = RSIIndicator(df_15min['Close']).rsi()
df_15min = df_15min.assign(rsi=rsi.values)

# Add Vol SMA 5
vol_sma5 = df_15min['Volume'].rolling(5).mean()
df_15min = df_15min.assign(vol_sma5=vol_sma5.values)

# Add ADX
ADX = ADXIndicator(high=df_15min['High'], low=df_15min['Low'], close=df_15min['Close'])
df_15min = df_15min.assign(adx=ADX.adx().values)
df_15min = df_15min.assign(adx_neg=ADX.adx_neg().values)
df_15min = df_15min.assign(adx_pos=ADX.adx_pos().values)

adx_sma5 = df_15min['adx'].rolling(5).mean()
df_15min = df_15min.assign(adx_sma5=adx_sma5.values)

adx_pos_sma5 = df_15min['adx_pos'].rolling(5).mean()
df_15min = df_15min.assign(adx_pos_sma5=adx_pos_sma5.values)

adx_neg_sma5 = df_15min['adx_neg'].rolling(5).mean()
df_15min = df_15min.assign(adx_neg_sma5=adx_neg_sma5.values)

df_15min.tail()


/usr/local/lib/python3.7/dist-packages/ta/trend.py:769: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ta/trend.py:774: RuntimeWarning:

invalid value encountered in double_scalars



,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos,adx_sma5,adx_pos_sma5,adx_neg_sma5
Date,,,,,,,,,,,,,
2021-12-17 14:15:00+05:30,462.25,462.40,460.5,460.95,31199,47.494622,40321.6,14.022258,24.751012,25.360386,14.910357,26.605628,22.337308
2021-12-17 14:30:00+05:30,460.85,461.30,458.3,459.00,52099,41.753209,44970.4,13.900628,29.386859,22.940435,14.605019,26.266693,23.541255
2021-12-17 14:45:00+05:30,459.00,460.25,457.2,460.00,100596,45.398475,59222.8,14.208369,30.018617,20.770435,14.466435,25.178060,25.084613
2021-12-17 15:00:00+05:30,460.00,461.40,459.0,459.65,143385,44.352248,73402.0,13.900477,27.790916,22.784975,14.207817,23.778793,26.587958
2021-12-17 15:15:00+05:30,459.70,459.75,458.0,458.00,161820,39.706588,97819.8,14.012145,29.407195,21.530309,14.008775,22.677308,28.270920


## Set Strategy Parameters

In [46]:
# Strategy params
strategy = 'RSIBuy'
sym = 'CADILAHC'
window_start = '2021-01-01 00:00:00'
window_end = '2021-12-15 00:00:00'
rsi_15min = 70
rsi_60min = 50
lot_size = 1100
stop_loss = 6500
stop_gain = 13000
back_candles = 5
adx_low = 30
adx_high = 40
volume_multiple = 2 # 2 or 3 times
daily_movement_pct = 4
check_volume = False


## Actual Strategy Implementation

### Step 1: A 15 min candle has to close above 70 RSI value
Get all the candles in the window that close above required RSI value

In [47]:
curr_window_df = df_15min[window_start:window_end]

rsi_filter = (curr_window_df['rsi'] > rsi_15min)
df = curr_window_df[rsi_filter]

bullish_filter = (df['Close'] > df['Open'])
df = df[bullish_filter]

temp_df = pd.DataFrame(columns=df_15min.columns)
temp_df.index.name = 'Date'
back_candles_df = pd.DataFrame()

for index, row in df.iterrows():
    back_candles_df = get_previous_candles(curr_window_df, index, 1)
    prev_rsi = 0
    if (back_candles_df.shape[0] != 0):
        prev_rsi = back_candles_df.iloc[0]['rsi']

    if (prev_rsi < rsi_15min):
        temp_df.loc[index] = row

df = temp_df[df_15min.columns]
df.head()


,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos,adx_sma5,adx_pos_sma5,adx_neg_sma5
Date,,,,,,,,,,,,,
2021-01-08 09:30:00+05:30,487.30,489.40,486.70,488.70,220463.0,71.494776,193226.2,13.684872,16.688542,32.242649,11.936672,24.450063,20.677380
2021-01-14 14:15:00+05:30,492.60,502.00,492.55,500.60,998004.0,76.603522,247846.8,13.322579,13.448315,42.395465,11.112032,26.188616,19.187833
2021-02-02 13:30:00+05:30,465.70,466.75,463.75,466.65,157785.0,70.998542,199070.0,26.316667,10.004965,29.145810,23.108763,26.527370,11.990670
2021-02-02 14:00:00+05:30,466.20,467.90,465.30,466.80,69357.0,70.414862,180988.2,30.219276,8.542732,29.129214,26.433340,29.500667,10.015765
2021-02-02 15:15:00+05:30,466.45,470.90,466.20,469.40,236930.0,73.437272,161363.0,35.982837,9.461806,31.848665,34.031644,27.781981,9.482578



### Step 4: Volume check

In [99]:
if (check_volume == True):
    temp_df = pd.DataFrame(columns=df_15min.columns)
    temp_df.index.name = 'Date'

    for index, row in df.iterrows():
        # Compare volume against previous mean volume
        if(row['Volume'] > volume_multiple*row['vol_sma5']):
            temp_df.loc[index] = row

    df = temp_df[df_15min.columns]
    df.head()

### Step 5: Hourly candle RSI check

In [65]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
    df_60min = get_hourly_df(df_15min, index)
    # print(df_60min.tail(15))

    rsi = df_60min.iloc[-1]['rsi']
    if(rsi > rsi_60min):
        # This 15min candle is eligible for signal
        # print('Hourly candle RSI is greater than 50..', rsi)
        temp_df.loc[index] = row

df = temp_df
df.head()


,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos
Date,,,,,,,,,,
2021-01-01 10:15:00+05:30,2407.10,2418.50,2407.10,2414.95,18458.0,62.760362,25412.6,17.606184,17.233116,25.627294
2021-01-01 13:15:00+05:30,2410.25,2416.05,2409.05,2415.00,10542.0,60.160375,8043.8,16.764313,13.366535,22.697455
2021-01-01 13:45:00+05:30,2414.15,2421.00,2413.55,2420.50,14861.0,64.302357,21749.0,21.020925,10.570558,29.824546
2021-01-04 09:30:00+05:30,2418.35,2424.50,2415.00,2422.30,20064.0,60.249955,28390.8,26.703770,14.266540,24.433453
2021-01-05 09:15:00+05:30,2405.00,2435.00,2387.70,2433.05,167707.0,69.504502,70279.0,15.730158,12.922847,29.101155


### Step 6: Stock movement check

In [66]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
    tanaji_pct = get_tanaji_pct(df_15min, index, 5, row['High'])
    # print('tanaji_pct:', tanaji_pct)
    if(tanaji_pct < daily_movement_pct):
        temp_df.loc[index] = row

df = temp_df
df.head()

,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos
Date,,,,,,,,,,
2021-01-01 10:15:00+05:30,2407.10,2418.50,2407.10,2414.95,18458.0,62.760362,25412.6,17.606184,17.233116,25.627294
2021-01-01 13:15:00+05:30,2410.25,2416.05,2409.05,2415.00,10542.0,60.160375,8043.8,16.764313,13.366535,22.697455
2021-01-01 13:45:00+05:30,2414.15,2421.00,2413.55,2420.50,14861.0,64.302357,21749.0,21.020925,10.570558,29.824546
2021-01-04 09:30:00+05:30,2418.35,2424.50,2415.00,2422.30,20064.0,60.249955,28390.8,26.703770,14.266540,24.433453
2021-01-05 09:15:00+05:30,2405.00,2435.00,2387.70,2433.05,167707.0,69.504502,70279.0,15.730158,12.922847,29.101155


### Step 7: Stop Loss check

In [48]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
    prev_candle = get_previous_candles(df_15min, index, 1)
    prev_low = prev_candle.iloc[0]['Low']
    if((row['High'] - prev_low)*lot_size > stop_loss):
        print(index, 'Stop Loss greater than 6000 INR. Do not trade', (row['High'] - prev_low)*lot_size)
        continue
    else:
        print(index, 'Stop Loss within range', (row['High'] - prev_low)*lot_size)
        temp_df.loc[index] = row

df = temp_df
df

2021-01-08 09:30:00+05:30 Stop Loss within range 6489.9999999999745
2021-01-14 14:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 13750.0
2021-02-02 13:30:00+05:30 Stop Loss within range 4730.000000000013
2021-02-02 14:00:00+05:30 Stop Loss within range 4234.999999999963
2021-02-02 15:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 6600.0
2021-02-05 12:15:00+05:30 Stop Loss within range 3189.999999999975
2021-02-12 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 7425.0
2021-02-25 10:15:00+05:30 Stop Loss within range 2750.0
2021-03-03 11:15:00+05:30 Stop Loss within range 5830.000000000013
2021-03-04 12:15:00+05:30 Stop Loss within range 4125.0
2021-03-04 12:45:00+05:30 Stop Loss within range 3685.000000000025
2021-03-22 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 9844.999999999987
2021-03-30 11:30:00+05:30 Stop Loss within range 3850.0
2021-03-30 15:15:00+05:30 Stop Loss within range 4894.999999999987
2021-03-31 11:00:00+05:30 Stop Los

,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos,adx_sma5,adx_pos_sma5,adx_neg_sma5
Date,,,,,,,,,,,,,
2021-01-08 09:30:00+05:30,487.30,489.40,486.70,488.70,220463.0,71.494776,193226.2,13.684872,16.688542,32.242649,11.936672,24.450063,20.677380
2021-02-02 13:30:00+05:30,465.70,466.75,463.75,466.65,157785.0,70.998542,199070.0,26.316667,10.004965,29.145810,23.108763,26.527370,11.990670
2021-02-02 14:00:00+05:30,466.20,467.90,465.30,466.80,69357.0,70.414862,180988.2,30.219276,8.542732,29.129214,26.433340,29.500667,10.015765
2021-02-05 12:15:00+05:30,483.80,486.00,483.15,485.10,243049.0,70.234326,200867.0,22.726262,11.320059,32.456896,18.891208,31.355283,13.361088
2021-02-25 10:15:00+05:30,440.30,442.50,439.35,442.50,188884.0,72.421022,275337.0,21.883846,13.685879,28.158950,20.398368,28.215438,15.600589
2021-03-03 11:15:00+05:30,442.45,447.55,441.80,445.50,204840.0,70.391956,74163.6,38.228958,7.481993,36.307410,35.523336,28.746615,9.347323
2021-03-04 12:15:00+05:30,448.85,450.75,448.40,450.45,107975.0,70.462410,75334.6,30.233592,10.848168,26.667205,29.525156,22.719682,11.922822
2021-03-04 12:45:00+05:30,450.15,453.00,450.15,452.30,191258.0,73.881134,100066.4,32.767290,9.328397,30.454428,30.503250,25.453894,11.014778
2021-03-30 11:30:00+05:30,430.05,433.50,429.50,433.15,85440.0,72.385272,88934.4,24.452396,12.944756,36.691562,21.445798,34.899599,15.043432


In [49]:
# ADX Check
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
    back_candles_df = get_previous_candles(curr_window_df, index, 1)
    prev_adx = back_candles_df.iloc[0]['adx']
    adx = row['adx']
    #if (adx >= adx_low and adx <= adx_high and adx - prev_adx >=0.75 and row['adx_pos'] > row['adx_neg'] and row['adx_pos'] >= row['adx_pos_sma5']):
    #    temp_df.loc[index] = row
    if (adx >= adx_low and adx <= adx_high):
        temp_df.loc[index] = row

df = temp_df
df

,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos,adx_sma5,adx_pos_sma5,adx_neg_sma5
Date,,,,,,,,,,,,,
2021-02-02 14:00:00+05:30,466.20,467.90,465.30,466.80,69357.0,70.414862,180988.2,30.219276,8.542732,29.129214,26.433340,29.500667,10.015765
2021-03-03 11:15:00+05:30,442.45,447.55,441.80,445.50,204840.0,70.391956,74163.6,38.228958,7.481993,36.307410,35.523336,28.746615,9.347323
2021-03-04 12:15:00+05:30,448.85,450.75,448.40,450.45,107975.0,70.462410,75334.6,30.233592,10.848168,26.667205,29.525156,22.719682,11.922822
2021-03-04 12:45:00+05:30,450.15,453.00,450.15,452.30,191258.0,73.881134,100066.4,32.767290,9.328397,30.454428,30.503250,25.453894,11.014778
2021-03-30 15:15:00+05:30,433.45,436.15,433.10,435.75,337261.0,71.721228,133525.8,31.738098,12.278784,27.514704,31.104431,26.747699,12.622109
2021-04-06 14:45:00+05:30,451.65,453.00,450.85,452.90,176699.0,72.420897,134151.0,31.609263,10.956365,29.067502,30.047424,26.558709,12.798118
2021-04-20 11:30:00+05:30,535.15,539.90,534.50,538.50,933547.0,73.227554,390914.0,32.549616,10.670221,33.275226,30.379618,30.311013,11.743663
2021-06-11 13:00:00+05:30,659.10,663.00,658.55,662.20,400932.0,70.889063,339526.4,31.955182,9.099681,33.589398,28.701049,29.072576,10.982597
2021-06-11 13:30:00+05:30,661.40,664.45,660.00,663.60,331735.0,71.474204,606446.6,36.208719,8.111423,33.572215,32.149744,33.196183,9.207613


## **Final Signal**

In [50]:
signals = list()
for index, row in df.iterrows():
    stop_loss_candle = get_previous_candles(df_15min, index, 1)
    sig_stop_loss = stop_loss_candle.iloc[0]['Low']
    sig_entry = row['High']
    sig_daily_mov_pct = np.round(get_tanaji_pct(df_15min, index, 5, row['High']), 2)
    sig_rsi = np.round(row['rsi'], 2)
    sig_hourly_rsi = np.round(get_hourly_df(df_15min, index).iloc[-1]['rsi'], 2)
    sig_mean_volume = row['vol_sma5']
    sig_volume = row['Volume']
    signal = Signal(strategy=strategy, sym=sym, lot_size=lot_size,
                    ts=index, entry_price=sig_entry, stop_loss=sig_stop_loss,
                    rsi=sig_rsi, hourly_rsi=sig_hourly_rsi, vol=sig_volume,
                    mean_vol=sig_mean_volume, daily_mov_pct=sig_daily_mov_pct)
    signals.append(signal)

for signal in signals:
    generate_buy_signal_pnl(signal, df_15min, stop_gain=stop_gain)

btResult = BackTestResult(sym=sym, signals=signals)

for signal in btResult.signals:
    print(signal)

print(btResult)

Strategy: RSIBuy, Sym: CADILAHC, TS: 2021-02-02 14:00:00+05:30, Entry: 467.9, StopLoss: 464.05, PnL: -4234.999999999963, Comment: Stop Loss Breached at 2021-02-02 14:45:00+05:30
Strategy: RSIBuy, Sym: CADILAHC, TS: 2021-03-03 11:15:00+05:30, Entry: 447.55, StopLoss: 442.25, PnL: -5830.000000000013, Comment: Stop Loss Breached at 2021-03-04 09:15:00+05:30
Strategy: RSIBuy, Sym: CADILAHC, TS: 2021-03-04 12:15:00+05:30, Entry: 450.75, StopLoss: 447.0, PnL: -4125.0, Comment: Stop Loss Breached at 2021-03-05 09:15:00+05:30
Strategy: RSIBuy, Sym: CADILAHC, TS: 2021-03-04 12:45:00+05:30, Entry: 453.0, StopLoss: 449.65, PnL: -3685.000000000025, Comment: Stop Loss Breached at 2021-03-04 13:30:00+05:30
Strategy: RSIBuy, Sym: CADILAHC, TS: 2021-03-30 15:15:00+05:30, Entry: 436.15, StopLoss: 431.7, PnL: 13000, Comment: Stop Gain reached at 2021-04-05 09:15:00+05:30
Strategy: RSIBuy, Sym: CADILAHC, TS: 2021-04-06 14:45:00+05:30, Entry: 453.0, StopLoss: 450.4, PnL: 13000, Comment: Stop Gain reached 

In [51]:
import plotly.graph_objects as go
import plotly.offline as py

back=5
next=50
sig_param_col1 = ['Time','Entry INR','StopLoss INR','Daily Mov %','Volume','Mean Vol','15Min RSI','Hourly RSI','PnL','Comments']
for signal in signals:
  prev_candles = get_previous_candles(df_15min, signal.ts, back, True)
  next_candles = get_next_candles(df_15min, signal.ts, next)
  
  candles = pd.concat([prev_candles, next_candles])
  candles['DateStr'] = candles.index.strftime('%d-%m %H:%M')

  # Get all strategy params for this signal

  sig_stop_loss = signal.stop_loss
  sig_entry = signal.entry_price
  sig_daily_mov_pct = signal.daily_mov_pct
  sig_rsi = signal.rsi
  sig_hourly_rsi = signal.hourly_rsi
  sig_mean_volume = signal.mean_vol
  sig_volume = signal.vol

  fig = make_subplots(rows=3, cols=2, shared_xaxes=False,
               subplot_titles=('OHLC', 'Signal Params', 'Volume & RSI','', 'ADX', ''), 
               vertical_spacing=0.1, 
               horizontal_spacing=0.01,
               row_width=[0.25, 0.25, 0.5],
               column_widths=[0.8,0.2],
               specs=[[{"secondary_y": False, "type": "candlestick"},{"secondary_y": False, "type":"table"}], 
                      [{"secondary_y": True}, {"secondary_y": False}],
                      [{"secondary_y": True}, {"secondary_y": False}]])
    
  fig.add_trace(go.Candlestick(x=candles['DateStr'],
                     open=candles['Open'],
                     high=candles['High'],
                     low=candles['Low'],
                     close=candles['Close'],
                     name='Signal Chart',
                     increasing_line_color='yellow',
                     increasing_fillcolor='yellow',
                     decreasing_line_color='red',
                     decreasing_fillcolor='red',),
                     row=1,col=1)
  
  fig.add_annotation(x=back,y=sig_entry,
                     text='Signal')
  
  # Position Entry Point
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=sig_entry, y1=sig_entry, 
                line=dict(color='Green'),
                row=1,col=1)
  
  # Position Stop Loss
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=sig_stop_loss, y1=sig_stop_loss, 
                line=dict(color='Red'),
                row=1,col=1)
  
  fig.add_annotation(x=back,y=sig_rsi,
                      text='Signal',row=2, col=1)
  
  fig.add_shape(type='line', x0=-1,x1=back+next+2,y0=rsi_15min, y1=rsi_15min, 
                line=dict(color='Green'),
                row=2,col=1)

  # Signal Parameters Table
  fig.add_trace(go.Table(header=dict(values=['Param','Value'],
                                     line_color='white',
                                     fill_color='darkslategray',
                                     align='left'),
                         cells=dict(values=[sig_param_col1,
                                           [signal.ts.strftime('%d-%m %H:%M'), sig_entry, 
                                            sig_stop_loss, sig_daily_mov_pct, sig_volume, 
                                            sig_mean_volume, sig_rsi, sig_hourly_rsi, signal.pnl, signal.comment]],
                                    line_color='white',
                                    fill_color='black',
                                    align='left')),
                row=1, col=2)

  # RSI
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['rsi'], name='rsi',            
           marker_color='Cyan'),
           row=2, col=1)
  
  # Volume
  fig.add_trace(go.Bar(x=candles['DateStr'], y=candles['Volume'], name='Volume',            
           marker_color='rgb(55, 55, 109)',
           width=np.array([0.5]*candles.size)),
           secondary_y=True,
          row=2, col=1)
  
  # ADX
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['adx'], name='adx',            
           marker_color='Cyan'),
           row=3, col=1)
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['adx_neg'], name='adx_neg',            
           marker_color='Blue'),
           row=3, col=1)
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['adx_pos'], name='adx_pos',            
           marker_color='Yellow'),
           row=3, col=1)

  fig.update_xaxes(type='category', rangeslider=dict(visible=False))
  fig.update_xaxes(showgrid=False, nticks=5)
  fig.update_yaxes(showgrid=False)
  fig.update_layout(
    title='Signal generated for RSI 15mins Strategy',
    title_x = 0.5,
    autosize=False,
    width=1450,
    height=850,
    plot_bgcolor='rgb(5,5,5)',
    paper_bgcolor='rgb(0,0,0)',
    font_color='white')

  py.iplot(fig)
  